In [31]:
### resdata - Jupyter Notebook analysis pipeline test
## The testing cells.
#1. Fetch single ring by id and visualise the data
#2. Fetch relevant data from full experiment and visualise as histogram
#3. Fetch relevant data from full experiment and visualise

In [38]:
## Sing in and Authentication
username = 'test_user_main'
password = 'some_password'
email = 'email@test_email.com'
full_name = 'Tomasz Neska'
path = 'http://127.0.0.1:8000/'
temp_file_name = "test.json"

In [39]:
# initialisation cell
import sys
sys.path.insert(0, "/home/splitsky/Desktop/code_repositories/resdata/")
import interface as ui
api = ui.API_interface(path)
api.check_connection()

True

In [40]:
# make user
result = api.create_user(username_in=username, password_in=password,email=email,full_name=full_name)
print(result)

False


In [42]:
# generate datasets and insert them into the database
import testing as t
api.generate_token(username, password)
#print(api.token)
# generate token
project_name = "project_test"
experiment_name = "experiment_test"
t.generate_optics_project(temp_file_name, [2,4],project_name, experiment_name, author_name=username)
project_from_file = t.load_file_project(filename_out=temp_file_name)
ui.insert(project(project_from_file))

AttributeError: 'str' object has no attribute 'get'

In [43]:
# print the files the user owns
api.tree_print()

The data tree:
